In [1]:
import Recomandation_models as rm
import Filtering as fl
import pandas as pd
import numpy as np

import math
from time import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torchvision 
import torch.utils.data as data
import torch.distributions as dist

### DATA PREPARATION

In [ ]:
df_books = pd.read_csv('books_autorec.csv')
print(len(df_books))
df_books.sort_values(by='ratings_count', ascending=False, inplace=True)
df_books = df_books.iloc[:1000]
df_books.to_csv('Spatial_model_books.csv')
book_ids = df_books['goodreads_book_id']
print(len(book_ids))

10000
1000


In [ ]:
df_books = pd.read_csv('Spatial_model_books.csv')

In [46]:
df = pd.read_csv("books_autorec.csv")
df_ratings = pd.read_csv("ratings_autorec.csv")

df_ratings_with_clusters = df_ratings.merge(
    df[['goodreads_book_id', 'cluster']], 
    left_on='book_id', 
    right_on='goodreads_book_id', 
    how='left'
)

# create dictionary with user and ratings
sparse_users = {}
for user_id, group in df_ratings_with_clusters.groupby('user_id'):
    books_ratings_clusters = group[['book_id', 'rating', 'cluster']].values.tolist()
    sparse_users[user_id] = books_ratings_clusters

cluster_sizes = df_books['cluster'].value_counts().sort_index().values

filter_users = {
    user: [triplet for triplet in triplets if triplet[0] in book_ids]
    for user, triplets in sparse_users.items()
}
filter_users = {user: triplets for user, triplets in filter_users.items() if triplets}
filter_users = sorted(filter_users.items(), key=lambda x: len(x[1]), reverse=True)[:10000]
filter_users = dict(filter_users)


In [47]:
print(len(sparse_users))
print(len(filter_users))

53424
10000


In [67]:
mapping_pos_to_books = dict(zip(range(1000), book_ids))
mapping_books_to_pos = dict(zip(book_ids,range(10000)))
mapping_pos_to_users = dict(zip(range(10000), filter_users.keys()))
mapping_users_to_pos = dict(zip(filter_users.keys(),range(10000)))
print(len(mapping_pos_to_books))
print(len(mapping_books_to_pos))
print(len(mapping_pos_to_users))
print(len(mapping_users_to_pos))

1000
1000
10000
10000


In [ ]:
n_books = len(mapping_books_to_pos)
user_vectors = []
for user_id, triplets in filter_users.items():
    vector = np.zeros(n_books)  # inizializza vettore di zeri

    for book_id, rating, _ in triplets:
        if book_id in mapping_books_to_pos:  # se il book_id è tra quelli mappati
            index = mapping_books_to_pos[book_id]
            vector[index] = rating  # inserisci il rating nella posizione giusta

    user_vectors.append(vector)
print(len(user_vectors))
print(len(user_vectors[0]))
df_input_data = pd.DataFrame(user_vectors)

10000
1000


In [ ]:
#df_input_data = pd.read_csv('Spatial_model_inputs.csv')
tensor_data = torch.tensor(df_input_data.values, dtype=torch.float32)
dataset = torch.utils.data.TensorDataset(tensor_data)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)